In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from metpy.units import units
from metpy import calc as mpcalc
import matplotlib
import matplotlib.pyplot as plt
from pathlib import Path
import math
import netCDF4
from netCDF4 import Dataset
from datetime import datetime, timedelta
from cftime import num2date, date2num
matplotlib.use('Agg')
import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
gh_file = "/home11/grad/2021/gd673524/research/gdines/data/ECMWF_gh_01252021.nc"
t_file = "/home11/grad/2021/gd673524/research/gdines/data/ECMWF_t_01252021.nc"
u_file = "/home11/grad/2021/gd673524/research/gdines/data/ECMWF_u_01252021.nc"
v_file = "/home11/grad/2021/gd673524/research/gdines/data/ECMWF_v_01252021.nc"
w_file = "/home11/grad/2021/gd673524/research/gdines/data/ECMWF_w_01252021.nc"

gh = Dataset(gh_file, mode='r')
t = Dataset(t_file, mode='r')
u = Dataset(u_file, mode='r')
v = Dataset(v_file, mode='r')
w = Dataset(w_file, mode='r')

In [3]:
gh

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2022-01-13 17:31:35 GMT by grib_to_netcdf-2.24.0: grib_to_netcdf /data/scratch/20220113-1720/6c/_mars-webmars-public-svc-green-003-6fe5cac1a363ec1525f54343b6cc9fd8-VNBmP5.grib -o /data/scratch/20220113-1720/5d/_grib2netcdf-webmars-public-svc-green-006-6fe5cac1a363ec1525f54343b6cc9fd8-jX9zk5.nc -utime
    dimensions(sizes): longitude(240), latitude(121), level(10), number(10), time(940)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 level(level), int32 number(number), int32 time(time), int16 gh(time, number, level, latitude, longitude)
    groups: 

In [26]:
#Calculating the Ensemble Means
z_ensmn = np.mean(gh.variables['gh'], axis=1)
t_ensmn = np.mean(t.variables['t'], axis=1)
u_ensmn = np.mean(u.variables['u'], axis=1)
v_ensmn = np.mean(v.variables['v'], axis=1)
w_ensmn = np.mean(w.variables['w'], axis=1)


In [20]:
#Creating Variables
lat = gh.variables['latitude'][:]
lon = gh.variables['longitude'][:]
level = gh.variables['level'][:] #<--Levels=[10,50,100,200,300,500,700,850,925,1000]
time = gh.variables['time'][:]
#time = np.asarray(times) <--unmasks array
t_1loc = t_ensmn[:,5,1,1]
#z_1loc = 
z_500 = np.asarray(z_ensmn[1,5,:,:])
t_500 = np.asarray(t_ensmn[1,5,:,:])

In [21]:
#Dealing with Time Variable
dates = [datetime(1900,1,1)+n*timedelta(hours=24) for n in range(time.shape[0])]

In [25]:
fig = plt.figure(figsize=(15,12))
fig, ax = plt.subplots(1, 1)

# Create a time series of 500hPa Temperatures for 1 location 
ax.plot(dates, t_1loc, color='tab:red')
ax.set_title('Ensemble Mean 500-hPa Temps at 1 pt')
ax.set_xlabel ('Time')
ax.set_ylabel ('Temp (K)')
ax.set_ylim(215,250)
ax.tick_params(axis='x', labelrotation = 90)

plt.show()

Ignore below this point! Haven't finished working out issues.

In [ ]:
plt.Figure(figsize=(15,15),dpi=120)    ## <---Set fig size here!

#Set plot as an orthographic projection looking down at the Earth from the pole with 0˚ at 6-oclock
ax = plt.axes(projection=ccrs.Orthographic(0,90))     
ax.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='dimgray',facecolor='none')
#ax.outline_patch.set_edgecolor('none')
gl = ax.gridlines(color="grey",linestyle=":",linewidth=0.5)  
glevs = range(300,600,60)  ## <---set contour interval for geo. heights
tlevs = range(0,400,4)        ## <--- set contour interval for temp
[x,y] = np.meshgrid(lon,lat)  ## <--- set lat/lon as coords on grid
        
plt.title("500-hPa Geopotential Height & Temp")
hghtplot = ax.contour(x,y,z_500,glevs,transform=ccrs.PlateCarree(),extend='both',colors='black',linewidth=1.0)
ax.clabel(hghtplot, inline=True, fmt="%.0f", fontsize=8)
tempplot = ax.contourf(x,y,t_500,tlevs,transform=ccrs.PlateCarree(),cmap="cool")
plt.colorbar(tempplot)

#plot_label = times[d].strftime("%Y%m%d%H")
plt.show()


In [ ]:
# Areal extent
latN = 37
latS = 26
lonW = -105
lonE = -92
latRange = np.arange(latS-5,latN+5,.25) # expand the data range a bit beyond the plot range
lonRange = np.arange((lonW-5+360),(lonE+5+360),.25) # Need to match longitude values to those of the coordinate variable

In [ ]:
#Creating Map projection and setting resolution
cLon = -95
cLat = 31.5
proj_map = ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)
proj_data = ccrs.PlateCarree()
res = '50m'

fig = plt.figure(figsize=(15,15))
ax = plt.subplot(1,1,1,projection=proj_map)
ax.set_extent ([lonW,lonE,latS,latN])
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))
CF = ax.contourf(lon,lat,t_500, tlevs,transform=proj_data,cmap=plt.get_cmap('coolwarm'))
cbar = plt.colorbar(CF,shrink=0.5)
cbar.ax.tick_params(labelsize=16)
#cbar.ax.set_ylabel(f"500-hPa Heights and Temperatures",fontsize=16)
CL = ax.contour(lon,lat,z_500,glevs,transform=proj_data,linewidths=1.25,colors='black')
ax.clabel(CL, inline_spacing=0.2, fontsize=11)

title = plt.title(f"500-hPa Heights and Temperatures",fontsize=16)
fig.tight_layout()
fig.show()